# Fetching observations from MET data base services

This notebook serves as sandbox for fetching data from frost.met, havvarsel-frost.met and thredds.met

See https://api.met.no/ for all interfaces and possible sources!!


## Havvarsel frost
Havvarsel frost delivers so far `temperature` measurements originated from badevann.no at seven beaches in the south of Norway (glider data integrated in the meantime, but not yet used here!). 
> Documentation:
> API documentation for obs/badevann https://havvarsel-frost.met.no/docs/apiref#/obs%2Fbadevann/obsBadevannGet 
> Datastructure described on https://havvarsel-frost.met.no/docs/dataset_badevann


## Frost
From the frost server we retrieve observations from the `n` closest weather observation stations and include the values 
- `air_temperature`
- `mean(surface_downwelling_shortwave_flux_in_air PT1H)`
- `wind_speed`
- `relative_humidity`
- `sum(duration_of_sunshinePT1H)`
- `cloud_area_fraction` (which takes values from 0 = no clouds to 8 = fully clouded) 
- ...

> Documentation:
> API documentation for observations on https://frost.met.no/api.html#!/observations/observations 
> Available elements (params) are listed on https://frost.met.no/elementtable 
> Examples on Frost data manipulation with Python on https://frost.met.no/python_example.html
>
> See also:
> Complete documentation at https://frost.met.no/howto.html 
> Complete frost API reference at https://frost.met.no/api.html 

## Thredds
Holds netcdf files with the a bunch of different data

> See the catalog: https://thredds.met.no/thredds/catalog.html

We primarily use the `Ocean and Ice/met.no (OLD) ROMS NorKyst800m coastal forecasting system` to get the forecasted water temperatures


TODO processing:
 - Tune processing and storing of observational data sets (to suite whatever code that will use the data sets)

In [104]:
# Importing general libraries
import sys
import json
import datetime
import requests
from traceback import format_exc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import netCDF4
import pyproj as proj


In [5]:
filename = "https://thredds.met.no/" + "/thredds/dodsC/metpparchive/2021/09/20/met_forecast_1_0km_nordic_20210920T06Z.nc"
filename

'https://thredds.met.no//thredds/dodsC/metpparchive/2021/09/20/met_forecast_1_0km_nordic_20210920T06Z.nc'

In [6]:
nc = netCDF4.Dataset(filename)

In [14]:
times = netCDF4.num2date(nc.variables["time"][:], nc.variables["time"].units)

In [17]:
nc.variables

{'forecast_reference_time': <class 'netCDF4._netCDF4.Variable'>
 float64 forecast_reference_time()
     units: seconds since 1970-01-01 00:00:00 +00:00
     standard_name: forecast_reference_time
 unlimited dimensions: 
 current shape = ()
 filling off,
 'projection_lcc': <class 'netCDF4._netCDF4.Variable'>
 int32 projection_lcc()
     grid_mapping_name: lambert_conformal_conic
     standard_parallel: [63. 63.]
     longitude_of_central_meridian: 15.0
     latitude_of_projection_origin: 63.0
     earth_radius: 6371000.0
     proj4: +proj=lcc +lat_0=63 +lon_0=15 +lat_1=63 +lat_2=63 +no_defs +R=6.371e+06
 unlimited dimensions: 
 current shape = ()
 filling off,
 'time': <class 'netCDF4._netCDF4.Variable'>
 float64 time(time)
     long_name: time
     standard_name: time
     units: seconds since 1970-01-01 00:00:00 +00:00
     _ChunkSizes: 1
 unlimited dimensions: time
 current shape = (59,)
 filling off,
 'x': <class 'netCDF4._netCDF4.Variable'>
 float64 x(x)
     standard_name: project

In [32]:
temps = nc["air_temperature_2m"][:,0,0][0:6]

In [33]:
len(temps)

6

In [38]:
filenames = ["https://thredds.met.no/thredds/dodsC/metpparchive/2021/09/20/met_forecast_1_0km_nordic_20210920T00Z.nc","https://thredds.met.no/thredds/dodsC/metpparchive/2021/09/20/met_forecast_1_0km_nordic_20210920T06Z.nc"]


In [39]:
ncs = netCDF4.MFDataset(filenames)

In [51]:
all_times = ncs.variables["time"]
all_times

<class 'netCDF4._netCDF4._Variable'>
float64 time('time',)
    long_name: time
    standard_name: time
    units: seconds since 1970-01-01 00:00:00 +00:00
    _ChunkSizes: 1
unlimited dimensions = ('time',)
current size = (118,)

In [43]:
temps = ncs["air_temperature_2m"][:,0,0]

In [44]:
len(temps)

118

In [45]:
first = netCDF4.num2date(all_times[0],all_times.units)
print("First available time: " + first.strftime("%Y-%m-%dT%H:%M"))

First available time: 2021-09-20T00:00


In [46]:
last = netCDF4.num2date(all_times[-1],all_times.units)
print("Last available time: " + last.strftime("%Y-%m-%dT%H:%M"))

Last available time: 2021-09-22T16:00


In [53]:
start_time = datetime.datetime.strptime("2021-09-20T00:00", "%Y-%m-%dT%H:%M")
end_time = datetime.datetime.strptime("2021-09-20T23:59", "%Y-%m-%dT%H:%M")

In [54]:
t1 = netCDF4.date2index(start_time, all_times, select="before")

In [56]:
t2 = netCDF4.date2index(end_time, all_times, select="after")

In [61]:
lat = 59.933329
lon = 10.7166638

In [102]:
proj_args = ncs.variables["projection_lcc"].proj4


In [103]:
proj_args = str(proj_args.replace("+no_defs",""))

In [105]:
p = proj.Proj(proj_args)

In [107]:
xp, yp = p(lon, lat)

In [110]:
lats = ncs.variables["latitude"][:]
lons = ncs.variables["longitude"][:]

xps, yps = p(lons, lats)

In [116]:
x = (np.abs(xps[0,:] - xp)).argmin()
y = (np.abs(yps[:,0] - yp)).argmin()

In [117]:
data = ncs.variables["air_temperature_2m"][t1:t2,y,x]


In [118]:
data

array([282.6989 , 282.63757, 282.5628 , 282.60977, 282.5229 , 282.22266,
       282.38187, 282.99582, 283.70224, 284.47678, 284.6016 , 284.67767,
       284.6973 , 284.8469 , 285.08008, 284.86697, 284.77582, 284.65115,
       284.19028, 284.2589 , 284.17197, 284.01154, 283.73694, 283.5991 ],
      dtype=float32)